In [1]:
import pandas as pd

# READ DATASETS

In [15]:
folder = "./datasets/"
output_folder = "./datasets_models/"

### STATIONS DATASET


In [3]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [4]:

df_mesures = pd.read_csv(folder + "datos_precip.csv", header=0).drop(['Unnamed: 0'], axis=1)

In [5]:
df_mesures

,id,codi_estacio,codi_variable,data_lectura,valor_lectura
0,CA350101090000,CA,35,2009-01-01T00:00:00.000,0.0
1,CC350101090000,CC,35,2009-01-01T00:00:00.000,0.0
2,CD350101090000,CD,35,2009-01-01T00:00:00.000,0.0
3,CE350101090000,CE,35,2009-01-01T00:00:00.000,0.0
4,CG350101090000,CG,35,2009-01-01T00:00:00.000,0.0
...,...,...,...,...,...
22437293,Z7352905231200,Z7,35,2023-05-29T12:00:00.000,0.0
22437294,Z8352905231200,Z8,35,2023-05-29T12:00:00.000,0.7
22437295,Z9352905231200,Z9,35,2023-05-29T12:00:00.000,0.0
22437296,ZB352905231200,ZB,35,2023-05-29T12:00:00.000,0.0


# PROCESS DATA

In [7]:
codiE_por_comarca = {}
for comarca, group in df_stations.groupby('comarca'):
    codiE_por_comarca[comarca] = list(group['codiE'].unique())

In [8]:
comarcas_bcn = ['Vallès Oriental', 'Alt Penedès', 'Baix Llobregat', 'Barcelonès', 'Garraf', 'Maresme', 'Vallès Occidental']
codiE_comarca = [codiE_por_comarca[i] for i in comarcas_bcn]

In [9]:
data_comarca = []
for i in comarcas_bcn:
    df_filtrado = df_mesures[df_mesures['codi_estacio'].isin(codiE_por_comarca[i])]
    data_comarca.append(df_filtrado.reset_index(drop=True))

In [10]:
for comarca in data_comarca:
    comarca.data_lectura = comarca.data_lectura.str.split('T').str[0]

In [11]:
media_por_dia_comarca = []
for comarca in data_comarca:
    comarca['data_lectura'] = pd.to_datetime(comarca['data_lectura'])
    media_por_dia_comarca.append(comarca.groupby([comarca['data_lectura'].dt.date, 'codi_variable'])['valor_lectura'].mean().reset_index())

In [12]:
nuevo_orden = ["comarca", "codi_variable", "data_lectura", "valor_lectura"]
for i in range(len(media_por_dia_comarca)):
    media_por_dia_comarca[i]["comarca"] = comarcas_bcn[i]
    media_por_dia_comarca[i] = media_por_dia_comarca[i][nuevo_orden].reset_index(drop=True)

In [16]:
df_concatenado = pd.concat(media_por_dia_comarca).reset_index(drop=True)
df_concatenado.to_csv(output_folder + "mean_precip_x_mes.csv", index=False)

In [14]:
df_concatenado

,comarca,codi_variable,data_lectura,valor_lectura
0,Vallès Oriental,35,2009-01-01,0.000000
1,Vallès Oriental,35,2009-01-02,0.008333
2,Vallès Oriental,35,2009-01-03,0.000000
3,Vallès Oriental,35,2009-01-04,0.002083
4,Vallès Oriental,35,2009-01-05,0.002083
...,...,...,...,...
37095,Vallès Occidental,35,2023-07-05,0.007143
37096,Vallès Occidental,35,2023-07-06,0.011905
37097,Vallès Occidental,35,2023-07-07,0.000000
37098,Vallès Occidental,35,2023-07-08,0.000000
